# Titre du notebook

*David Scanu*

---
## Sommaire : 
- Objectif
- Description des colonnes
- Importer les bibliothèques
- Importer les données
- Analyse statistique de base
- Visualisation
- Préparation des données
  - Nettoyage
  - Outliers
  - Equilibre
- Séparation des données
- Feature selection
- Mise à l'échelle
- Machine learning
- Export du modèle

---

## Notre objectif

Décrire notre objectif : classer, prédire, ...

## Description des caractères / colonnes : 

- Caractère 1
- Caractère 2
- ...

## Importer les bibliothèques

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Importer les données

In [ ]:
# Importe les données
data = pd.read_csv() # Dataframe

In [ ]:
# Montre les premières lignes du jeu de données
data.head()

## Analyse statistique de base

### Moyennes et médianes

In [ ]:
# data.groupby().agg(['mean', 'median'])

### Ecart-type par espèces

In [ ]:
data.groupby().std()

## Visualisation

### Distribution

In [ ]:
# Boxplot
sns.set(style="ticks") 
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
sns.boxplot(x='',y='',data=data)
plt.subplot(2,2,2)
sns.boxplot(x='',y='',data=data)
plt.subplot(2,2,3)
sns.boxplot(x='',y='',data=data)
plt.subplot(2,2,4)
sns.boxplot(x='',y='',data=data)
plt.show()

In [ ]:
# Violin Plot
# Montre la distribution des données sur plusieurs niveaux
# d'une (ou plusieurs) variables catégorielles de sorte que
# ces distributions puissent être comparées.

sns.set(style="whitegrid")
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
sns.violinplot(x='', y='',data=data)
plt.subplot(2,2,2)
sns.violinplot(x='',y='',data=data)
plt.subplot(2,2,3)
sns.violinplot(x='',y='',data=data)
plt.subplot(2,2,4)
sns.violinplot(x='',y='',data=data)
plt.show()

### Pairplot

Premier reflexe, tracer un pairplot pour mettre en évidence répartitions et corrélations.

In [ ]:
sns.pairplot(data=data, hue="target", height=3)

### Analyse univariée

### Analyse Bivariée

#### Tableau de correlations

In [ ]:
# Affiche les correlations
data.corr()

#### Heatmap

In [ ]:
# Visualise les correlations
sns.heatmap(data.corr(), annot=True, cmap='RdBu')

#### Scatter Plot

In [ ]:
# Visualise l'impact de deux variables sur la cible
sns.scatterplot(data=data, x='', y='', hue='target')
plt.title('Titre')
plt.show()

## Préparations des données

### Nettoyage

Vérifier les valeurs manquantes dans le jeu de données.

In [ ]:
print(data.isnull().values.any())

In [ ]:
print(data.isnull().sum())

*Stratégie 1* : Suppression des lignes contenant des valeurs manquantes.

In [ ]:
data_no_nan = data.dropna()

*Stratégie 2* : Remplacer les valeurs manquantes par : 
- Moyenne
- Médiane
- Mode
- Valeur arbitraire
- ...

*Selon la stratégie*

In [ ]:
# SimpleImputer
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(data)

# imputer.transform retourne un ndarray, donc besoin de recréer un DataFrame
data_no_nan_mode = pd.DataFrame(imputer.transform(df_nan.loc[:,:]), columns = data.columns)
data_no_nan_mode.head()

### Outliers

- Visualiser les outliers avec boxplot()
- Enlever les outliers

### Equilibre

Vérifier que les cibles du jeu de données sont **bien équilibrées**.

In [ ]:
data['target'].value_counts()

### Encoder les données

Encoder les valeurs qui sont des catégories, en valeurs numériques :

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
data['target'] = labelencoder.fit_transform(data['target'])

print(data['target'])
print(labelencoder.classes_)

## Séparation des données

Séparer le jeu de données :
- X : les variables
- y : la cible

In [ ]:
# Fonctionne avec DataFrame ou ndarray

# Variables
X = # Enlever la colonne 'target' du DataFrame

# Cible
y = data['target'] # Garder uniquement la colonne cible



Séparer le jeu de données en données d'entrainement et de test :
- X_train : données d'entrainement
- X_test : données de test
- y_train : cible d'entrainement
- y_test : cible de test

In [ ]:
from sklearn.model_selection import train_test_split

# Split les données, retourne DataFrame ou ndarray
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

print(f"Nombre d'exemples d'entrainement X : {X_train.shape[0]}")
print(f"Nombre d'exemples de test X : {X_test.shape[0]}")
print(' ')
print(f"Nombre d'exemples d'entrainement Y : {y_train.shape[0]}")
print(f"Nombre d'exemples de test Y : {y_test.shape[0]}")

## Feature Selection

#### VarianceThreshold

Elimine les variables dont la **variance est inférieur à un certain seuil**.

In [ ]:
# Variance des variables
X_train.var(axis=0)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector_vt = VarianceThreshold(threshold=0.2)
selector_vt.fit(X_train)
# Affiche un masque
selector_vt.get_support()


In [ ]:
# Affiche les colonnes restantes
np.array(X_train.columns)[selector_vt.get_support()]

In [ ]:
X_train_vt = selector_vt.transform(X_train)
X_test_vt = selector_vt.transform(X_test)
X_train_vt[:10] # Affiche les 10 premières lignes de notre ndarray

#### SelectKbest

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Retourne 2 tableaux :
# - score test chi2, dépendance à y
# - P values
chi2(X, y)

In [ ]:
selector_kb = SelectKBest(chi2, k=2)
selector_kb.fit(X_train, y)
selector_kb.get_support() # Retourne une seule variable/colonne

In [ ]:
# Affiche les colonnes restantes
np.array(X_train.columns)[selector_kb.get_support()]

In [ ]:
X_train_kb = selector_kb.transform(X_train)
X_test_kb = selector_kb.transform(X_test)
X_train_kb[:10] # Affiche les 10 première ligne de notre ndarray

#### SelectFromModel

Entraine un estimateur puis selectionne les **variables** les plus importantes pour cet estimateur. Compatible avec les estimateurs qui développent une fonction paramétrée *(Ne fonctionne pas avec Knn)*.

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import SGDClassifier

selector_sfm = SelectFromModel(SGDClassifier(random_state=0), threshold='mean')
selector_sfm.fit(X_train, y_train)
selector_sfm.get_support()

In [ ]:
# Affiche les colonnes restantes
np.array(X_train.columns)[selector_sfm.get_support()]

In [ ]:
X_train_sfm = selector_sfm.transform(X_train)
X_test_sfm = selector_sfm.transform(X_test)
X_train_sfm[:10]

In [ ]:
# Matrice coefficient de 3 x 4
selector_sfm.estimator_.coef_

In [ ]:
# Le selecteur selectionne les variables > à ce chiffre
selector_sfm.estimator_.coef_.mean(axis=0).mean()

#### RFE + RFECV

Eliminent les variables les moins importantes de façon **récursive**.

Un estimateur est entrainé plusieurs fois, après chaque entrainement, des features sont éliminées sur la base de **coefficients** les plus faibles de l'estimateur.



In [ ]:
from sklearn.feature_selection import RFE, RFECV

selector_RFECV = RFECV(SGDClassifier(random_state=0), step=1, min_features_to_select=2, cv=5)
selector_RFECV.fit(X_train, y_train)
selector_RFECV.ranking_


In [ ]:
selector_RFECV.cv_results_

In [ ]:
selector_RFECV.get_support()

In [ ]:
# Affiche les colonnes restantes
np.array(X_train.columns)[selector_RFECV.get_support()]

In [ ]:
X_train_RFECV = selector_RFECV.transform(X_train)
X_test_RFECV = selector_RFECV.transform(X_test)
X_train_RFECV[:10]

## Mise à l'échelle

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_kb)
X_test_scaled = scaler.transform(X_test_kb)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train_minmaxscaler = min_max_scaler.fit_transform(X_train_kb)
X_test_minmaxscaler = min_max_scaler.transform(X_test_kb)

In [ ]:
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()
X_train_robscaler = robust_scaler.fit_transform(X_train_kb)
X_test_robscaler = robust_scaler.transform(X_test_kb)

## Machine Learning

### Modèle KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Nous entrainons notre modèle avec les données d'entrainements (standardisées)
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train_scaled, y_train)
print('Train Score : ', model.score(X_train_scaled, y_train))
print('Test Score : ', model.score(X_test_scaled, y_test))

### Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut, StratifiedShuffleSplit

# Choix de la méthode de validation croisée (échantillonage)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
# cv = LeaveOneOut()

cross_val_res = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
print(cross_val_res)

In [ ]:
cross_val_res.mean()

In [ ]:
# Cross-validation for each k value
val_score = []
for k in range(1, 50):
    score = cross_val_score(KNeighborsClassifier(n_neighbors=k), X_train_scaled, y_train, cv=5).mean()
    val_score.append(score)
plt.plot(val_score)

### Validation Curve

In [ ]:
from sklearn.model_selection import validation_curve

k = np.arange(1, 50)
train_score, val_score = validation_curve(KNeighborsClassifier(), X_train_scaled, y_train, param_name='n_neighbors', param_range=k, cv=5)

plt.plot(k, val_score.mean(axis=1), label="Validation")
plt.plot(k, train_score.mean(axis=1), label="Train")
plt.ylabel('score')
plt.xlabel('n_neighbors')
plt.legend()
plt.show()

### Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors' : np.arange(1, 20), 'metric' : ['euclidean', 'manhattan']}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train_scaled, y_train)

Afficher le meilleur score

In [ ]:
grid.best_score_

Afficher les paramètres donnant les meilleures performances

In [ ]:
grid.best_params_

Sauvegarder le meilleur modèle

In [ ]:
# Best model
model_best = grid.best_estimator_

Tester le nouveau modèle sur les données de test

In [ ]:
# Score du meilleur modèle
model_best.score(X_test_scaled, y_test)

### Matrice de confusion

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model_best.predict(X_test_scaled)
confusion_matrix = confusion_matrix(y_test, y_pred)


In [ ]:
cm_data = {'prediction': y_pred, 'actual': y_test}
cm_df = pd.DataFrame(cm_data)
contingency_matrix = pd.crosstab(cm_df['prediction'], cm_df['actual'])
print(contingency_matrix)

In [ ]:
sns.heatmap(contingency_matrix.T, annot=True, fmt='.2f', cmap="YlGnBu", cbar=False)
plt.title('Matrice de confusion')
plt.show()

#### Learning Curve

In [ ]:
from sklearn.model_selection import learning_curve

N, train_score, val_score = learning_curve(model_best, X_train_scaled, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

print(N)
plt.plot(N, train_score.mean(axis=1), label='train')
plt.plot(N, val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()
plt.show()

## Export du modèle

Exporter le modèle avec joblib ou Pickle. Il faut exporter :
- Le modèle
- Le scaler
- le nom des colonnes X
- les valeurs possibles de la 'target' (si catégorie)

In [ ]:
import pickle

dict_export = {}
dict_export['model'] = model_best
dict_export['scaler'] = scaler
dict_export['X_col_names'] = X_col_names
dict_export['y_names'] = data.target_names

pickle_out = open("model.pkl","wb")
pickle.dump(dict_export, pickle_out)
pickle_out.close()